In [22]:
from src.utils.data_utils import load_dataframe_from_csv
from src.data.dataloader import load_initial_dataset
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
movie, character, plot_summaries,  tvtropes, name_clusters = load_initial_dataset('data')
oscar_winning_films = load_dataframe_from_csv('oscar_winning_films_ids')
oscar_winning_actors = load_dataframe_from_csv('oscar_winning_actors')
oscar_winning_actresses = load_dataframe_from_csv('oscar_winning_actresses')
oscar_supporting_actors = load_dataframe_from_csv('oscar_winning_supporting_actors')
oscar_supporting_actresses = load_dataframe_from_csv('oscar_winning_supporting_actresses')
extended_films = load_dataframe_from_csv('film_2015_2024.csv')
academy_award_winning_films = load_dataframe_from_csv('acedemy_award_winning_films')